# Braiding on an Ising chain

We perform braiding on the Ising in the following manner:
- first we initialize the coupler by application of an Rx(π/2) gate, and the ferro magnetic chain in $|\uparrow\uparrow\uparrow\rangle$ (ie $|000\rangle$)
- next we adiabatically transfer the ferro portion from the left hand side to the right hand-side of the coupler
- next we apply a Ry(π) pulse on the coupler and transfer the chain back
- at the end of the procedure we expect to bet in the $|\downarrow\downarrow\downarrow\rangle$ (ie $|111\rangle$)

In [259]:
import numpy as np
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister
from ising_kitaev import initialize_chain, braid_chain, rotate_to_measurement_basis, add_measurement, initialize_coupler

## Setup

### Parameters

We consider a chain with 12 sites and 1 coupler. The 3 leftmost sites start in a ferromagnetic state.

In [260]:
zeeman_ferro = 0.01
zeeman_para = 10

In [261]:
initial_config = np.array([zeeman_ferro]*3 + [zeeman_para]*9)

### Initialize the circuit

We initialize the system in $|\uparrow\uparrow\uparrow\rangle$ (ie $|000\rangle$) since the braiding will induce a phase gate in the logical space defined by:

\begin{align}
|0\rangle = \frac{|\uparrow\uparrow\uparrow\rangle + |\downarrow\downarrow\downarrow\rangle}{\sqrt{2}}\qquad
|1\rangle = \frac{|\uparrow\uparrow\uparrow\rangle - |\downarrow\downarrow\downarrow\rangle}{\sqrt{2}}
\end{align}

In [262]:
qreg = QuantumRegister(13)
creg = ClassicalRegister(3)
qcirc = QuantumCircuit(qreg, creg)

In [263]:
initialize_chain(qcirc, qreg, initial_config, 'up')
initialize_coupler(qcirc, qreg)
qcirc.draw()

### Braid the chain

We evolve the Zeeman slowly and in between each update of the Zeeman we run the dynamic of the chain using Suzuki-Trotter decomposition for t such that J*t = 1

In [264]:
braid_chain(qcirc, qreg, np.pi, 40, initial_config, 1.4, 0.25, 0.25, 2, 40, method='both')
qcirc.depth()

137281

### Add the measurement

In [265]:
add_measurement(qcirc, qreg, creg, [0, 1, 2])

## Run the simulation

In [266]:
from qiskit import Aer, execute

In [267]:
backend = Aer.get_backend('qasm_simulator')
job = execute(qcirc, backend, shots=2000)

In [268]:
job.status()

<JobStatus.RUNNING: 'job is actively running'>

At the end of the procedure we observe a transfer in $|\downarrow\downarrow\downarrow\rangle$ (ie $|111\rangle$) with a probability exiding 75\%.

It should be noted that the transfer probability is sensitivitive to the choice of the evolution of the system.

In [269]:
result = job.result()
print(result.get_counts())

{'000': 213, '100': 7, '110': 6, '011': 18, '001': 2, '111': 1754}
